In [1]:
!pip install requests
import requests
import json
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

def float_to_string(f):
  f = np.round(f,4)
  s = str(f)[2:]
  while len(s) < 4:
    s += "0"
  return s

Mounted at /content/drive


# Collect betting data

In [7]:
odds = pd.read_csv('https://raw.githubusercontent.com/d141/epl_predictions/main/NHL_Odds_Data/nhl%20odds%2018.csv')
odds_team_names = list(odds.Team.unique())
odds.head()

,Date,Rot,VH,Team,1st,2nd,3rd,Final,Open,Close,Puck Line,Unnamed: 11,Open OU,Unnamed: 13,Close OU,Unnamed: 15
0,1003,1,V,Montreal,1,1,0,2,184,210,1.5,-125,6.0,-110,6.0,-110
1,1003,2,H,Toronto,1,1,0,3,-220,-240,-1.5,105,6.0,-110,6.0,-110
2,1003,3,V,Boston,0,0,0,0,115,105,1.5,-290,5.5,-110,5.5,-120
3,1003,4,H,Washington,2,4,1,7,-135,-115,-1.5,230,5.5,-110,5.5,100
4,1003,5,V,Calgary,0,0,2,2,-130,-130,-1.5,200,5.5,-110,5.5,-115


In [55]:
string = "https://statsapi.web.nhl.com/api/v1/game/2018020001/linescore"
response = requests.get(string)
game = json.loads(response.content.decode('utf-8'))

In [56]:
game['currentPeriod']

4

In [57]:
from datetime import datetime

start = game['periods'][3]['startTime']
print(start)
datetime.strptime(game['periods'][0]['startTime'], "%Y-%m-%dT%H:%M:%Sz")

2018-10-04T01:51:27Z


datetime.datetime(2018, 10, 3, 23, 16, 14)

# Collect game history

In [ ]:
#https://statsapi.web.nhl.com/api/v1/game/2018020001/linescore
#Use above for start time and OT or not OT
#convert zulu time to eastern, then get the date, otherwise a late start will end up in the next day

end_reached = False
year='2019'
game_number = .0001
list_2018 = []

while not end_reached:

  try:
    game_string = float_to_string(game_number)
    boxscore_string = f"https://statsapi.web.nhl.com/api/v1/game/{year}02{game_string}/boxscore"
    boxscore_response = requests.get(boxscore_string)
  except:
    #Reached the end of the season
    end_reached = True
    break

  game = json.loads(boxscore_response.content.decode('utf-8'))
  teams=['away', 'home']

  for team in teams:
    try:
      players = list(game['teams'][team]['players'].keys())
    except:
      end_reached = True
      print(game_number, game_string)
      break
    stats = []
    for player in players:
      try:
        game['teams'][team]['players'][player]['stats']['skaterStats']['player_id'] = player
        stats.append(game['teams'][team]['players'][player]['stats']['skaterStats'])
      except:
        try:
          #Player is a goalie or is scratched
          game['teams'][team]['players'][player]['stats']['goalieStats']['player_id'] = player
          stats.append(game['teams'][team]['players'][player]['stats']['goalieStats'])
        except:
          #Player scratched
          continue
    df = pd.DataFrame.from_dict(stats)
    df = df.set_index('player_id')
    df[f'{team}_team_name'] = game['teams'][team]['team']['name']
    df[f'{team}_team_id'] = game['teams'][team]['team']['id']
    df['season'] = year
    df['game_number'] = game_number
    list_2018.append(df)
  game_number += .0001

In [ ]:
df_2018 = pd.concat(list_2018)

In [ ]:
df_2018.sample(10)

In [ ]:
len(df_2018)

In [34]:
from copy import Error
def process_box_scores(df):

  df['game_number'] = df['game_number'] * 10000
  df['game_number'] = df['game_number'].astype(int)

  home_teams = df_2018[df_2018['away_team_name'].isna()]
  away_teams = df_2018[df_2018['home_team_name'].isna()]
  by_game_away = away_teams.groupby(['game_number']).sum()
  by_game_home = home_teams.groupby(['game_number']).sum()

  num_games = by_game_away.index.argmax()

  if num_games != by_game_home.index.argmax():
    print("Error: Number of home and away games don't match")
    return False

  season = []

  for i in range(1,num_games+1,1):
    box_score = {}
    this_game_all = df[df['game_number'] == i]
    this_game_home = this_game_all[this_game_all['away_team_name'].isna()]
    this_game_away = this_game_all[this_game_all['home_team_name'].isna()]
    box_score['game_number'] = i
    try:
      box_score['home_team_name'] = this_game_home.home_team_name.iloc[0]
      box_score['away_team_name'] = this_game_away.away_team_name.iloc[0]
    except:
      print(f"Error getting the team names in game {i}")
    #print(by_game_home)
    try:
      box_score['home_goals'] = this_game_home.goals.sum()
      box_score['away_goals'] = this_game_away.goals.sum()
      box_score['home_shots'] = this_game_home[this_game_home.evenSaves.isna()].shots.sum()
      box_score['away_shots'] = this_game_away[this_game_away.evenSaves.isna()].shots.sum()
      box_score['home_hits'] = this_game_home.hits.sum()
      box_score['away_hits'] = this_game_away.hits.sum()
      box_score['home_ppg'] = this_game_home.powerPlayGoals.sum()
      box_score['away_ppg'] = this_game_away.powerPlayGoals.sum()
      box_score['home_goals'] = this_game_home.goals.sum()
      box_score['away_goals'] = this_game_away.goals.sum()
    except:
      print("Error getting goals")
      print()
      return False
    season.append(box_score)

  return pd.DataFrame(season)
  
def get_linescore_data(df, year):
  df['start_time'] = ""
  df['periods_played'] = 0
  for i in range(len(df)):
    game_string = float_to_string((df.loc[i].game_number)/10000)
    linescore_string = f"https://statsapi.web.nhl.com/api/v1/game/{year}02{game_string}/linescore"
    linescore_response = requests.get(linescore_string)
    linescore = json.loads(linescore_response.content.decode('utf-8'))

    num_periods = int(linescore['currentPeriod'])

    #for i in range(num_periods):
    #  period = linescore['periods'][0]
    #  period_string = f"{period + 1}"
    #  df[f""]

    df.at[i, 'start_time'] = datetime.strptime(linescore['periods'][0]['startTime'], "%Y-%m-%dT%H:%M:%Sz")
    df.at[i, 'periods_played'] = num_periods
  return df

In [35]:
df_2018 = pd.read_csv('/content/drive/MyDrive/2018_box_scores.csv')
new_df = process_box_scores(df_2018)
df2 = get_linescore_data(new_df, "2018")

Error getting the team names in game 24
Error getting the team names in game 48
Error getting the team names in game 93
Error getting the team names in game 356


In [69]:
df2.sample(10)

,game_number,home_team_name,away_team_name,home_goals,away_goals,home_shots,away_shots,home_hits,away_hits,home_ppg,away_ppg,start_time,periods_played
357,358,Arizona Coyotes,Calgary Flames,1,6,29,27,31.0,10.0,0.0,1.0,2018-11-25 20:10:57,3
651,652,Toronto Maple Leafs,Nashville Predators,0,4,18,32,18.0,31.0,0.0,1.0,2019-01-08 00:08:13,3
309,310,Montréal Canadiens,Washington Capitals,4,5,44,34,20.0,27.0,1.0,1.0,2018-11-20 00:08:02,4
30,31,Boston Bruins,Ottawa Senators,6,3,30,31,18.0,17.0,1.0,0.0,2018-10-08 02:38:14,3
15,16,Los Angeles Kings,San Jose Sharks,2,3,21,33,34.0,21.0,0.0,0.0,2018-10-06 02:47:47,4
182,183,Detroit Red Wings,New Jersey Devils,4,3,31,28,25.0,20.0,2.0,2.0,2018-11-01 23:38:34,3
874,875,Pittsburgh Penguins,Edmonton Oilers,3,1,34,39,51.0,41.0,0.0,0.0,2019-02-14 01:10:02,3
1095,1096,Colorado Avalanche,Anaheim Ducks,3,5,44,37,31.0,36.0,1.0,1.0,2019-03-16 01:12:26,3
387,388,Edmonton Oilers,Los Angeles Kings,3,2,34,32,35.0,27.0,2.0,1.0,2018-11-30 02:07:17,3
702,703,Vancouver Canucks,Florida Panthers,5,1,36,24,34.0,18.0,0.0,0.0,2019-01-14 00:11:18,3
